In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysia-hansard-qa/resolve/main/hansard-qa.json

In [14]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [3]:
with open('hansard-qa.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

42440

In [5]:
data[0]

[{'original': '\n22                                                            DR.30.10.2018                                                                                                                          \n\n \ndan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya \nmasuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita.  [Disampuk]  \n\nTimbalan Yang di-Pertua [Dato’ Mohd Rashid Hasnon]: Yang Berhormat Padang \nSerai, sila ajukan soalan. \n\nTuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak \nkementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. \n[Dewan tepuk] Terima kasih. \n\nTuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. \n\n■1150 \n\nTuan Chong Chieng Jen: Itu satu isu atau soalan yang begitu serius dan effect-\nnya yang sangat luas. Saya rasa ini perlu dibawa ke dalam Jemaah Menteri untuk \ndipertimbangkan dan diputuskan. Sekian, terima kasih. \n\n \n\n

In [15]:
data_parlimen = []
for d in data:
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        data_parlimen.append(
            {
                'context': t['original'],
                'qa': j_a
            }
        )
    except:
        pass
    
len(data_parlimen)

42368

In [8]:
indices = {i for i in range(len(data_parlimen))}

In [9]:
data_parlimen[0]['qa']['soalan']

[{'soalan': 'Adakah samsu akan dihapuskan terus dari negara kita?',
  'jawapan': 'Ini perlu dibawa ke dalam Jemaah Menteri untuk dipertimbangkan dan diputuskan.'},
 {'soalan': 'Adakah kerajaan bercadang untuk menyemak semula jadual perjalanan bagi kereta api komuter dan ETS dari Gemas ke Kuala Lumpur?',
  'jawapan': 'KTMB pada masa ini masih belum ada perancangan untuk menyemak semula jadual perjalanan bagi perkhidmatan komuter dan ETS daripada Gemas ke Kuala Lumpur.'},
 {'soalan': 'Berapa banyak set ETS yang dimiliki oleh KTMB untuk tujuan operasi?',
  'jawapan': 'KTMB memiliki 15 set train ETS yang aktif untuk tujuan operasi.'}]

In [20]:
from tqdm import tqdm

pairs = []
data_qa = data_parlimen
for i in tqdm(range(len(data_qa))):
    query = data_qa[i]['context']
    
    for s in data_qa[i]['qa']['soalan']:
        pos = [query]
        
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data_qa[k]['context'])
                if overlapped < 0.1:
                    neg.append(data_qa[k]['context'])
                if len(neg) >= 5:
                    break
            except:
                pass
        
        pairs.append({
            'query': s['soalan'],
            'pos': pos,
            'neg': neg
        })

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42368/42368 [1:18:39<00:00,  8.98it/s]


In [22]:
len(pairs)

127114

In [24]:
with open('hansard-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [25]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='hansard-qa-pair.jsonl',
    path_in_repo='hansard-qa-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

hansard-qa-pair.jsonl:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/05d04fb50063d7ace53462071c52e0c6cd61ae0e', commit_message='Upload hansard-qa-pair.jsonl with huggingface_hub', commit_description='', oid='05d04fb50063d7ace53462071c52e0c6cd61ae0e', pr_url=None, pr_revision=None, pr_num=None)